In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from scipy import constants
from scipy import stats
from sklearn.linear_model import LinearRegression

In [63]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


***

# 2. IDADE

- De acordo com os relatos, as primeiras pessoas que embarcaram nos botes salva-vidas foram mulheres e crianças. A coluna 'Sex' está completa, mas a coluna 'Age' está com múltiplos valores faltando;
- Primeiro passo é completar essas linhas com valores, principalmente para crianças do sexo masculino;

### 2.1. ENCONTRAR QUAIS DOS PASSAGEIROS SEM IDADE SÃO CRIANÇAS DO SEXO MASCULINO

`Separar quais pessoas não têm registro de idade e são do sexo masculino:`

In [65]:
age = df[df['Age'].isna()]
maleAge = age[age['Sex']=="male"]

`Identificar qual o Index dos meninos sem idade registrada:`

In [66]:
maleKids = maleAge[maleAge['Name'].str.contains("Master.", regex=False)]
listaGarotos = maleKids.index

- Apenas 4 pessoas das que não possuem idade registrada são crianças e do sexo masculino (assim como Mrs. é utilizado para se referir à uma senhora, Master. é utilizado para se referir à um menino);
- Ainda há diversas idades em branco, falta definir uma media a seguir para essas idades;
- Além disso, há idades com casas decimais: se for menor que 1 ano e para idades estimadas. Arredondar todas para cima e transformar em int.

### 2.2. ATRIBUINDO IDADE MEDIA (SELECIONANDO O MELHOR MÉTODO) PARA AS MULHERES

`Media por Mrs. ou Miss.:`

In [67]:
mulheres = df[df['Sex']=='female']
avrg_Mrs = round(mulheres[mulheres['Name'].str.contains('Mrs.')]['Age'].mean())
avrg_Miss = round(mulheres[mulheres['Name'].str.contains('Miss.')]['Age'].mean())
avrg_Mrs, avrg_Miss

(36, 22)

`Media por classe do ticket:`

In [68]:
avrg_m1 = round(mulheres[mulheres['Pclass']==1]['Age'].mean())
avrg_m2 = round(mulheres[mulheres['Pclass']==2]['Age'].mean())
avrg_m3 = round(mulheres[mulheres['Pclass']==3]['Age'].mean())
avrg_m1, avrg_m2, avrg_m3

(35, 29, 22)

`Escolhida media por classe do ticket, criar lista com Index das mulheres sem idade separadas por classe de ticket:`

In [69]:
mulheres_sem_idade = age[age['Sex']=="female"]
mulheres_1class = mulheres_sem_idade[mulheres_sem_idade['Pclass']==1].index
mulheres_2class = mulheres_sem_idade[mulheres_sem_idade['Pclass']==2].index
mulheres_3class = mulheres_sem_idade[mulheres_sem_idade['Pclass']==3].index

`Medias inseridas nos valores em branco para as mulheres:`

In [70]:
df.loc[mulheres_1class,'Age']=avrg_m1
df.loc[mulheres_2class,'Age']=avrg_m2
df.loc[mulheres_3class,'Age']=avrg_m3

### 2.3. ATRIBUINDO IDADE MEDIA (SELECIONANDO O MELHOR MÉTODO) PARA OS HOMENS

`Media por Mr. ou Master.:`

In [71]:
homens = df[df['Sex']=='male']
avrg_Mr = round(homens[homens['Name'].str.contains('Mr.')]['Age'].mean())
avrg_Master = round(homens[homens['Name'].str.contains('Master.')]['Age'].mean())
avrg_Mr, avrg_Master

(32, 5)

`Media por classe do ticket:`

In [72]:
avrg_h1 = round(homens[homens['Pclass']==1]['Age'].mean())
avrg_h2 = round(homens[homens['Pclass']==2]['Age'].mean())
avrg_h3 = round(homens[homens['Pclass']==3]['Age'].mean())
avrg_h1, avrg_h2, avrg_h3

(41, 31, 27)

`Escolhida media por classe do ticket, criar lista com Index dos homens sem idade separados por classe de ticket:`

In [73]:
homens_1class = maleAge[maleAge['Pclass']==1].index
homens_2class = maleAge[maleAge['Pclass']==2].index
homens_3class = maleAge[maleAge['Pclass']==3].index

`Medias inseridas nos valores em branco para os homens (para meninos, media utilizado foi avrg_Master):`

In [74]:
df.loc[homens_1class,'Age']=avrg_h1
df.loc[homens_2class,'Age']=avrg_h2
df.loc[homens_3class,'Age']=avrg_h3
df.loc[listaGarotos,'Age']=avrg_Master

***

# 3. TRANSFORMANDO COLUNAS
- Agora, já sabemos todas as pessoas com prioridade de embarcar nos barcos salva-vidas: mulheres e crianças. Será criada uma coluna para identificar essas pessoas ('Priority'), em que o valor 1 as representa, e 0 representa os demais passageiros
- Transformar coluna 'Sex' para: 1 se male, e 2 se female
- Arredondar valores das colunas 'Age' e 'Fare' e transformá-las em int64 type
- Transformar coluna 'Embarked' para: 1 se C, 2 se Q, e 3 se S. Completar os dois valores faltando
- Retirar colunas desnecessárias ('Name', 'Ticket', 'Cabin')

### 3.1. DEFININDO A COLUNA 'PRIORITY'

`Colocar valor 1 para passageiros(as) com prioridade:`

In [75]:
priority = []
for row in df['Sex']:
    if row == 'female':
        priority.append(1)
    else: priority.append(0)

df['Priority'] = priority

df.loc[listaGarotos,'Priority']=1

### 3.2. ALTERANDO COLUNA 'SEX'

`Transformar 'male' e 'female' em 1 e 2, respectivamente:`

In [76]:
lista_homens = homens.index
lista_mulheres = mulheres.index
df.loc[lista_homens,'Sex']=int(1)
df.loc[lista_mulheres,'Sex']=int(2)
df = df.astype({'Sex':'int64'})

### 3.3. ALTERANDO COLUNAS 'AGE' E 'FARE'

`Arredondar idades e transformar em inteiros:`

In [77]:
df['Age'] = df['Age'].round()
df = df.astype({'Age':'int64'})

`Arredondar preço das passagens e transformar em inteiros:`

In [78]:
df['Fare'] = df['Fare'].round()
df = df.astype({'Fare':'int64'})

### 3.4. ALTERANDO COLUNA 'EMBARKED'

`Criar listas com separação por local de embarcação:`

In [79]:
lista_embC = df[df['Embarked']=='C'].index
lista_embQ = df[df['Embarked']=='Q'].index
lista_embS = df[df['Embarked']=='S'].index

`Completar valores vazios:`

In [80]:
embNA = df[df['Embarked'].isna()]
lista_embNA = df[df['Embarked'].isna()].index
df.loc[lista_embNA,'Embarked']=3

`Alterar valores na coluna:`

In [81]:
df.loc[lista_embC,'Embarked']=1
df.loc[lista_embQ,'Embarked']=2
df.loc[lista_embS,'Embarked']=3
df = df.astype({'Embarked':'int64'})

### 3.5. RETIRANDO COLUNAS DESNECESSÁRIAS PARA CALCULAR CORRELAÇÃO

`Retirar colunas 'Name', 'Ticket' e 'Cabin':`

In [82]:
df_int = df.drop(['PassengerId','Name','Ticket','Cabin'], inplace=False, axis=1)

***
# 4. CORRELAÇÃO
- Geração dos 3 métodos de correlação: Pearson, Kendall e Spearman
- Indicar quais features são as mais correlatas com coluna 'Survived'

### 4.1. MÉTODOS DE CORRELAÇÃO

`Pearson:`

In [83]:
df_int.corr(method='pearson').head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Priority
Survived,1.0,-0.338481,0.543351,-0.069939,-0.035322,0.081629,0.257088,-0.167675,0.544043


`Kendall:`

In [84]:
df_int.corr(method='kendall').head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Priority
Survived,1.0,-0.323533,0.543351,-0.040274,0.085915,0.133933,0.279197,-0.155747,0.544043


`Spearman:`

In [85]:
df_int.corr(method='spearman').head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Priority
Survived,1.0,-0.339668,0.543351,-0.048484,0.088879,0.138266,0.33135,-0.160196,0.544043


In [86]:
corr, p = stats.pearsonr(df_int.Survived,df_int.Priority)
print(corr)
print(p)

0.5440427919484425
8.737989754412024e-70


In [87]:
x = df_int[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Priority']]
y = df_int[['Survived']]
model = LinearRegression().fit(x,y)
r_sq = model.score(x,y)
intercept = model.intercept_
coeficientes = model.coef_
r_sq

0.4057522223442658